## Предсказание стоимости жилья

В рамках данного проекта необходимо обучить модель **линейной регрессии** на данных о **жилье в Калифорнии в 1990 году**. Для этого мне был предоставлен датасет, который содержит следующие данные:

- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

**Задача**: На основе данных нужно предсказать медианную стоимость дома в жилом массиве — **median_house_value**. Обучить модель и сделать предсказания на тестовой выборке. **Для оценки качества модели** будем использовать метрики **RMSE**, **MAE** и **R2**.

# Подготовка данных

### Знакомство с данными

In [2]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

In [3]:
# Инициализируем Spark сессию

spark = SparkSession.builder \
                    .master('local') \
                    .appName('California real estate 1990') \
                    .getOrCreate()

# Распакуем данные
data = spark.read.csv('/datasets/housing.csv', inferSchema = True, header=True)

# Выведем данные DF
data.show(3)
print('+-------------------+')
print(f'| Всего строк {data.count()} |')
print('+-------------------+')

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 3 rows

+--------

In [4]:
# Выведем данные колонок, а также их описание
display(data.describe().toPandas())
print('----------------------------')
pd.DataFrame(data.dtypes, columns=['column', 'type']).head(10)

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


----------------------------


,column,type
0,longitude,double
1,latitude,double
2,housing_median_age,double
3,total_rooms,double
4,total_bedrooms,double
5,population,double
6,households,double
7,median_income,double
8,median_house_value,double
9,ocean_proximity,string


#### Вывод "Знакомство с данными"

Нам достался небольшой DF на 20*К наблюдений и 10 столбцами, 1 из которых является нашим таргетом - median_house_value. Однако, если говорить про типы данных - необходимо double перевести в float, дабы сэкономить нашу память.

### Предобработка данных

In [5]:
# Изменим тип данных double на float

dtype_double = [col for col in data.columns]
del dtype_double[-1]

for col in dtype_double:
    data = data.withColumn(col, F.col(col).cast(FloatType()))
    
pd.DataFrame(data.dtypes, columns=['column', 'type']).head(10)

,column,type
0,longitude,float
1,latitude,float
2,housing_median_age,float
3,total_rooms,float
4,total_bedrooms,float
5,population,float
6,households,float
7,median_income,float
8,median_house_value,float
9,ocean_proximity,string


In [6]:
# Поработаем с пропусками

for col in data.columns:
    print(col, data.where(F.col(col).isNull() | F.isnan(col)).count(), data.where(F.col(col).isNull() | F.isnan(col)).count()/data.count())

longitude 0 0.0
latitude 0 0.0
housing_median_age 0 0.0
total_rooms 0 0.0
total_bedrooms 207 0.01002906976744186
population 0 0.0
households 0 0.0
median_income 0 0.0
median_house_value 0 0.0
ocean_proximity 0 0.0


In [7]:
# Удалим 1% nan значений
data = data.na.fill(537)

# Проверим результат
for col in data.columns:
    print(col, data.where(F.col(col).isNull() | F.isnan(col)).count(), data.where(F.col(col).isNull() | F.isnan(col)).count()/data.count())

longitude 0 0.0
latitude 0 0.0
housing_median_age 0 0.0
total_rooms 0 0.0
total_bedrooms 0 0.0
population 0 0.0
households 0 0.0
median_income 0 0.0
median_house_value 0 0.0
ocean_proximity 0 0.0


#### Вывод "Предобработка"

В результате предобработки данных были обнаружены nan значения в столбцах данных, однако они составляли всего 1%, поэтому мы их смело удалили. Также тип данных double был изменён на float, чтобы оптимизировать память.

# Обучение моделей

In [8]:
# Преобразуем колонку с категориальными значениями техникой One hot encoding

#Вначале применим StringIndexer, чтобы перевести категориальные в численные

indexer = StringIndexer(inputCol='ocean_proximity', 
                        outputCol='ocean_proximity_idx') 
data = indexer.fit(data).transform(data)

data.select(['ocean_proximity', 'ocean_proximity_idx']).show(3) 

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



In [9]:
# Затем разобьём численные на колонки методом OHE

encoder = OneHotEncoder(inputCol='ocean_proximity_idx',
                        outputCol='ocean_proximity_ohe')
data = encoder.fit(data).transform(data)

data.select(['ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe']).show(3) 

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



In [10]:
# Финальный шаг преобразований — это объединение признаков в один вектор, с которым ML-алгоритм умеет работать

categorical_cols = ['ocean_proximity']

categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                        outputCol='categorical_features')
data = categorical_assembler.transform(data) 

In [11]:
# Разделим выборку

train, test = data.randomSplit([.75,.25], seed=12345)
print(train.count(), test.count()) 

15406 5234


In [12]:
# Для числовых признаков тоже нужна трансформация — шкалирование значений — чтобы сильные выбросы не смещали предсказания модели

numerical_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']

numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train = numerical_assembler.transform(train) 
test = numerical_assembler.transform(test)

standardScaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
train = standardScaler.fit(train).transform(train)
test = standardScaler.fit(test).transform(test)

In [13]:
train.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'ocean_proximity_idx',
 'ocean_proximity_ohe',
 'categorical_features',
 'numerical_features',
 'numerical_features_scaled']

In [14]:
# Финальный шаг — собрать трансформированные категорийные и числовые признаки с помощью VectorAssembler

all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train = final_assembler.transform(train)
test = final_assembler.transform(test)

train.select(all_features).show(3) 
test.select(all_features).show(3) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-61.982223876749...|
|       (4,[2],[1.0])|     [-61.957303671685...|
|       (4,[2],[1.0])|     [-61.957303671685...|
+--------------------+-------------------------+
only showing top 3 rows

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-62.255949041483...|
|       (4,[2],[1.0])|     [-62.225878367535...|
|       (4,[2],[1.0])|     [-62.225878367535...|
+--------------------+-------------------------+
only showing top 3 rows



In [15]:
# Обучим модель

lr = LinearRegression(featuresCol = 'features',labelCol='median_house_value', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train)

22/10/24 16:00:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/10/24 16:00:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


# Анализ результатов

In [16]:
# Train Выборка все столбцы
trainingSummary = lr_model.summary

trainingSummary.residuals.show()
print("MAE: %f" % trainingSummary.meanAbsoluteError)
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

+-------------------+
|          residuals|
+-------------------+
| -129829.5888681477|
| -60489.64972118451|
| -78724.81692099618|
|-119658.66199386795|
| -95825.22369057732|
|-108033.64937744848|
| -179768.1786751626|
| -85635.97185885091|
| -78991.20949681196|
|-100608.44417862664|
| -83495.13357291277|
| -92372.32823971682|
|-125838.28158681653|
|-126506.27865370037|
|   -97470.480198059|
| -61822.67340289056|
| -97594.60929040262|
|-107428.61917272373|
|-103218.79416175047|
|-116024.68944024574|
+-------------------+
only showing top 20 rows

MAE: 50397.302620
RMSE: 69808.165558
r2: 0.633040


In [17]:
# Сравним с RMSE Данного столбца
train.describe().toPandas().median_house_value

0                15406
1    207063.3730364793
2    115242.0680417819
3              14999.0
4             500001.0
Name: median_house_value, dtype: object

In [18]:
# Test выборка все столбцы

predictions = lr_model.transform(test)

rmse = RegressionEvaluator(predictionCol='prediction', labelCol='median_house_value', metricName='rmse')
mae = RegressionEvaluator(predictionCol='prediction', labelCol='median_house_value', metricName='mae')
r2 = RegressionEvaluator(predictionCol='prediction', labelCol='median_house_value', metricName='r2')

print("MAE test: %f" % mae.evaluate(predictions))
print("RMSE test: %f" % rmse.evaluate(predictions))
print("r2 test: %f" % r2.evaluate(predictions))

MAE test: 50572.223496
RMSE test: 67960.367858
r2 test: 0.655838


In [19]:
# Train численные переменные

num_train = train.select(['numerical_features_scaled','median_house_value'])
num_test = test.select(['numerical_features_scaled','median_house_value'])
print(num_train.count(), num_test.count())

lr_num = LinearRegression(featuresCol = 'numerical_features_scaled', labelCol='median_house_value', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model_num = lr_num.fit(num_train)

15406 5234


In [20]:
# Train Выборка все столбцы
trainingSummarynum = lr_model_num.summary

trainingSummarynum.residuals.show()
print("MAE: %f" % trainingSummarynum.meanAbsoluteError)
print("RMSE: %f" % trainingSummarynum.rootMeanSquaredError)
print("r2: %f" % trainingSummarynum.r2)

+-------------------+
|          residuals|
+-------------------+
| -93527.49017469259|
| 12300.055955793709|
|-6420.7048864858225|
| -77667.26108106505|
| -56569.01372766914|
| -71562.66868724115|
|-135428.58651847718|
|-10884.265044343192|
|-10740.894353624899|
| -52545.20585463615|
|-11780.936154561117|
| -19968.91819633171|
| -80053.32844285434|
|  -80513.3973504263|
| -48633.28664713679|
|    11797.291323293|
|-53350.884250159375|
| -58571.27106578136|
| -54332.76444053603|
| -68406.46236593882|
+-------------------+
only showing top 20 rows

MAE: 51366.825141
RMSE: 70620.580533
r2: 0.624449


In [21]:
# Test выборка все столбцы

predictions = lr_model_num.transform(num_test)

rmse_t = RegressionEvaluator(predictionCol='prediction', labelCol='median_house_value', metricName='rmse')
mae_t = RegressionEvaluator(predictionCol='prediction', labelCol='median_house_value', metricName='mae')
r2_t = RegressionEvaluator(predictionCol='prediction', labelCol='median_house_value', metricName='r2')

print("MAE test only num: %f" % mae_t.evaluate(predictions))
print("RMSE test only num: %f" % rmse_t.evaluate(predictions))
print("r2 test only num: %f" % r2_t.evaluate(predictions))

MAE test only num: 54100.294031
RMSE test only num: 70135.659195
r2 test only num: 0.633453


In [22]:
# Остановим сессию Spark

spark.stop()

### Вывод

В результате обучения модели линейной регресси на всех данный были получены следующие результаты:
1. На train выборке:
    - RMSE = 69501.603498
    - MAE = 50292.996357
    - r2: 0.637912
2. На test выборке:
    - MAE test: 49801.420094
    - RMSE test: 68395.176993
    - r2 test: 0.647666
    
В результате обучения модели линейной регресси только на числовых значениях были получены следующие результаты:
1. На train выборке:
    - MAE: 51096.328830
    - RMSE: 70255.279685
    - r2: 0.630017
2. На test выборке:
    - MAE test only num: 51120.557433
    - RMSE test only num: 69597.442066
    - r2 test only num: 0.635170
    
Данную модель нельзя назвать сильной или даже хорошей, R2 = в среднем 64%, говорит нам о том, что лишь 64% данных возможно объяснить при помощи нашей модели, а средний разброс в 70000, с точки зрения всего значения является также довольно большой величиной.